In [ ]:
import pandas as pd
authors = pd.read_csv('author_profiles.csv')
authors.columns

In [ ]:
authors_sub = authors.loc[authors['enneagram_type'].notnull()]
authors_sub['enneagram_type'].unique()

In [ ]:
enne_dict = dict()
for ind, row in authors_sub.iterrows():
    if row['enneagram_type'] not in enne_dict:
        enne_dict[row['enneagram_type']] = 1
    else:
        enne_dict[row['enneagram_type']] += 1

In [ ]:
for k, v in enne_dict.items():
    print('For enneagram type ' + str(k) + ' there are ' + str(v) + ' users')

In [ ]:
authors_combo = pd.read_csv('all_authors_combined_liwc.csv')
authors_combo = authors_combo.loc[authors_combo['enneagram_type'].notnull()]
print(list(authors_combo.columns))

In [ ]:
from scipy.stats import pearsonr

# Test for correlation with Enneagram types
liwc_cols = ['WC', 'WPS', 'Sixltr', 'Dic', 'funct', 'pronoun', 'ppron', 'i', 'we', 'you', 'shehe', 'they', 'ipron', 'article', 'verb', 'auxverb', 'past', 'present', 'future', 'adverb', 'preps', 'conj', 'negate', 'quant', 'number', 'swear', 'social', 'family', 'friend', 'humans', 'affect', 'posemo', 'negemo', 'anx', 'anger', 'sad', 'cogmech', 'insight', 'cause', 'discrep', 'tentat', 'certain', 'inhib', 'incl', 'excl', 'percept', 'see', 'hear', 'feel', 'bio', 'body', 'health', 'sexual', 'ingest', 'relativ', 'motion', 'space', 'time', 'work', 'achieve', 'leisure', 'home', 'money', 'relig', 'death', 'assent', 'nonfl', 'filler', 'AllPunc', 'Period', 'Comma', 'Colon', 'SemiC', 'QMark', 'Exclam', 'Dash', 'Quote', 'Apostro', 'Parenth', 'OtherP']

for col in liwc_cols:
    results = pearsonr(authors_combo['enneagram_type'], authors_combo[col])
    if results[1] < 0.05:
        print('Correlation for ' + col + ' is ' + str(results[0]))

In [ ]:
#keys = comments['subreddit'].unique()
comments_dict = dict()
unique_users_dict = dict()
base_url = 'https://files.pushshift.io/reddit/comments/'
files = [
    'RC_2015-01.zst',
    'RC_2015-02.zst',
    'RC_2015-03.zst',
    'RC_2015-04.zst',
    'RC_2015-05.zst',
    'RC_2015-06.zst',
    'RC_2015-07.zst',
    'RC_2015-08.zst',
    'RC_2015-09.zst',
    'RC_2015-10.zst',
    'RC_2015-11.zst',
    'RC_2015-12.zst',
    'RC_2016-01.zst',
    'RC_2016-02.zst',
    'RC_2016-03.zst',
    'RC_2016-04.zst',
    'RC_2016-05.zst',
    'RC_2016-06.zst',
    'RC_2016-07.zst',
    'RC_2016-08.zst',
    'RC_2016-09.zst',
    'RC_2016-10.zst',
    'RC_2016-11.zst',
    'RC_2016-12.zst',
    'RC_2017-01.zst',
    'RC_2017-02.zst',
    'RC_2017-03.zst',
    'RC_2017-04.zst',
    'RC_2017-05.zst',
    'RC_2017-06.zst',
    'RC_2017-07.zst',
    'RC_2017-08.zst',
    'RC_2017-09.zst',
    'RC_2017-10.zst',
    'RC_2017-11.zst',
    'RC_2017-12.zst',
    'RC_2018-01.zst',
    'RC_2018-02.zst',
    'RC_2018-03.zst',
    'RC_2018-04.zst',
    'RC_2018-05.zst',
    'RC_2018-06.zst',
    'RC_2018-07.zst',
    'RC_2018-08.zst',
    'RC_2018-09.zst',
    'RC_2018-10.zst',
    'RC_2018-11.zst',
    'RC_2018-12.zst',
    'RC_2019-01.zst',
    'RC_2019-02.zst',
    'RC_2019-03.zst',
    'RC_2019-04.zst',
]

unique_users_dict = dict()

In [ ]:
import subprocess
import os
from tqdm import tqdm
import ijson
import sys
import pandas as pd
import glob
import os

# Batch iterate downlaoded pushshift to conserve memory
for file in tqdm(files):
    url = 'https://files.pushshift.io/reddit/comments/' + file
    subprocess.call(['wget',url,], stderr=subprocess.DEVNULL, stdout=subprocess.DEVNULL)
    subprocess.call(['unzstd',file,'--memory=2048MB'], stderr=subprocess.DEVNULL, stdout=subprocess.DEVNULL)
    with open(file.split('.')[0], 'rb') as f:
        for record in ijson.items(f, '', multiple_values=True):
            if record['subreddit'] not in comments_dict:
                comments_dict[record['subreddit']] = 1
                unique_users_dict[record['subreddit']] = set()
            else:
                comments_dict[record['subreddit']] += 1
            unique_users_dict[record['subreddit']].add(record['author'])
    dataf = pd.DataFrame({'subreddit': pd.Series(comments_dict.keys()),'comments_count':pd.Series(comments_dict.values()),'unique_authors':pd.Series(len(v) for v in unique_users_dict.values())})
    dataf.to_csv('reddit-summary/csv_dump-' + file.split('.')[0] +'.csv', index=False)
    for fl in glob.glob('./' + file):
        os.remove(fl)
    for fl in glob.glob('./' + file.split('.')[0]):
        os.remove(fl)

In [ ]:
from os import listdir
from os.path import isfile, join
reddit_path = './reddit-summary/'
onlyfiles = [f for f in listdir(reddit_path) if isfile(join(reddit_path, f))]
onlyfiles = sorted(onlyfiles)
onlyfiles.reverse()
new_path = './monthly-report/'
previous_name = onlyfiles[0]
for csvFile in onlyfiles[1:]:
    current = pd.read_csv(reddit_path + csvFile)
    previous = pd.read_csv(reddit_path + previous_name)

    diff = previous.copy()
    diff['comments_count'] = previous['comments_count'] - current['comments_count']
    diff['unique_authors'] = previous['unique_authors'] - current['unique_authors']
    diff.to_csv(new_path + previous_name, index=False)
    previous_name = csvFile

# Split collected cumulative data into per-month reports
last = pd.read_csv(reddit_path + onlyfiles[-1])
last.to_csv(new_path + onlyfiles[-1], index=False)

In [ ]:
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None

# Apply binning to big five traits

authors = pd.read_csv('./data/author_profiles.csv')
authors['agreeableness_median'] = np.nan
authors['conscientiousness_median'] = np.nan
authors['openness_median'] = np.nan
authors['neuroticism_median'] = np.nan
authors['extraversion_median'] = np.nan

authors['agreeableness_median'][authors.agreeableness <= authors.agreeableness.median()] = 0
authors['agreeableness_median'][authors.agreeableness > authors.agreeableness.median()] = 1

authors['conscientiousness_median'][authors.conscientiousness <= authors.agreeableness.median()] = 0
authors['conscientiousness_median'][authors.conscientiousness > authors.agreeableness.median()] = 1

authors['openness_median'][authors.openness <= authors.agreeableness.median()] = 0
authors['openness_median'][authors.openness > authors.agreeableness.median()] = 1

authors['neuroticism_median'][authors.neuroticism <= authors.agreeableness.median()] = 0
authors['neuroticism_median'][authors.neuroticism > authors.agreeableness.median()] = 1

authors['extraversion_median'][authors.extraversion <= authors.agreeableness.median()] = 0
authors['extraversion_median'][authors.extraversion > authors.agreeableness.median()] = 1

authors['agreeableness_quartile'] = np.nan
authors['conscientiousness_quartile'] = np.nan
authors['openness_quartile'] = np.nan
authors['neuroticism_quartile'] = np.nan
authors['extraversion_quartile'] = np.nan

authors['agreeableness_quartile'][authors.agreeableness <= authors.agreeableness.quantile(25)] = 0
authors['agreeableness_quartile'][(authors.agreeableness > authors.agreeableness.quantile(25)) & (authors.agreeableness < authors.agreeableness.quantile(75))] = 1
authors['agreeableness_quartile'][authors.agreeableness >= authors.agreeableness.quantile(75)] = 2

authors['conscientiousness_quartile'][authors.conscientiousness <= authors.conscientiousness.quantile(25)] = 0
authors['conscientiousness_quartile'][(authors.conscientiousness > authors.conscientiousness.quantile(25)) & (authors.conscientiousness < authors.conscientiousness.quantile(75))] = 1
authors['conscientiousness_quartile'][authors.conscientiousness >= authors.conscientiousness.quantile(75)] = 2

authors['openness_quartile'][authors.openness <= authors.openness.quantile(25)] = 0
authors['openness_quartile'][(authors.openness > authors.openness.quantile(25)) & (authors.openness < authors.openness.quantile(75))] = 1
authors['openness_quartile'][authors.openness >= authors.openness.quantile(75)] = 2

authors['neuroticism_quartile'][authors.neuroticism <= authors.neuroticism.quantile(25)] = 0
authors['neuroticism_quartile'][(authors.neuroticism > authors.neuroticism.quantile(25)) & (authors.neuroticism < authors.neuroticism.quantile(75))] = 1
authors['neuroticism_quartile'][authors.neuroticism >= authors.neuroticism.quantile(75)] = 2

authors['extraversion_quartile'][authors.extraversion <= authors.extraversion.quantile(25)] = 0
authors['extraversion_quartile'][(authors.extraversion > authors.extraversion.quantile(25)) & (authors.extraversion < authors.extraversion.quantile(75))] = 1
authors['extraversion_quartile'][authors.extraversion >= authors.extraversion.quantile(75)] = 2

authors.to_csv('./data/author_profiles_extended_normal.csv')

In [ ]:
import pandas as pd
import re
import numpy as np
# Add new values (previously binned)
files = ['./res/agreeableness_median-LR-NP/preds.csv', './res/conscientiousness_median-LR-NP/preds.csv', './res/extraversion_median-LR-NP/preds.csv', './res/neuroticism_median-LR-NP/preds.csv', './res/openness_median-LR-NP/preds.csv']
dim = ['agreeableness_median', 'conscientiousness_median', 'extraversion_median', 'neuroticism_median', 'openness_median']
data = pd.read_csv('./data/mbti_enne_pred.csv')
for file, dimension in zip(files, dim):
    preds = pd.read_csv(file)
    preds[dimension + '_higher'] = preds.confidence.apply(lambda x: ['0.' + s for s in re.findall(r'\\d+', x)][-1])
    preds[dimension + '_lower'] = preds.confidence.apply(lambda x: ['0.' + s for s in re.findall(r'\\d+', x)][-5])
    merger = preds[['author', dimension + '_higher', dimension + '_lower']]
    data = pd.merge(data, merger, how='inner', on = 'author')



files = ['./res/agreeableness_quartile-LR-NP/preds.csv', './res/conscientiousness_quartile-LR-NP/preds.csv', './res/extraversion_quartile-LR-NP/preds.csv', './res/neuroticism_quartile-LR-NP/preds.csv', './res/openness_quartile-LR-NP/preds.csv']
dim = ['agreeableness_quartile', 'conscientiousness_quartile', 'extraversion_quartile', 'neuroticism_quartile', 'openness_quartile']
for file, dimension in zip(files, dim):
    preds = pd.read_csv(file)
    preds[dimension + '_lower'] = preds.confidence.apply(lambda x: ['0.' + s for s in re.findall(r'\\d+', x)][-9])
    preds[dimension + '_middle'] = preds.confidence.apply(lambda x: ['0.' + s for s in re.findall(r'\\d+', x)][-5])
    preds[dimension + '_higher'] = preds.confidence.apply(lambda x: ['0.' + s for s in re.findall(r'\\d+', x)][-1])
    merger = preds[['author', dimension + '_lower', dimension + '_middle', dimension + '_higher']]
    data = pd.merge(data, merger, how='inner', on = 'author')
    

data.to_csv('./data/mbti_enne_pred_extended_normal.csv')